In [1]:
import folium
import geopandas as gpd
import pandas as pd

In [37]:
# Import the shapefile with data
gdf = gpd.read_file('data/Census_Tracts_2020_Redistricting_Data.shp')
# Import the labels that I gave to the tracts 
labels = pd.read_csv('data/tract_labels.csv').drop(['TRACT_NO', 'FIPS2020'], axis=1)
# Cast types to enable merge
gdf['STATE'] = gdf['STATE'].astype('int64')
gdf['COUNTY'] = gdf['COUNTY'].astype('int64')
gdf['TRACT'] = gdf['TRACT'].astype('int64')
# Merge the shapefile with the labels
gdf = gdf.merge(labels, on=['STATE', 'COUNTY', 'TRACT'], how='left')
gdf = gdf[gdf['LABEL']!='Not Portland']
gdf = gdf[gdf['LABEL'].notnull()]
gdf[gdf['LABEL'].notnull()].head(2)

,OBJECTID,STATE,COUNTY,TRACT,TRACT_NO,FIPS2020,P2_001N,P2_002N,P2_005N,P2_006N,...,P2_009N,P2_010N,P2_011N,GISACRES,Shape__Are,Shape__Len,geometry,POPULATION,LABEL,LABEL_NOTE
210,211,41,51,2401,24.01,41051002401,2794,173,2096,175,...,4,18,224,208.084401,9.064165e+06,12542.520021,"POLYGON ((-122.65860 45.54827, -122.65768 45.5...",2794.0,NE,inner NE
211,212,41,51,2402,24.02,41051002402,3830,308,2905,180,...,4,17,254,222.753325,9.703143e+06,13945.775924,"POLYGON ((-122.65760 45.53435, -122.65760 45.5...",3830.0,NE,inner NE


In [20]:
# Create a map and add choropleth by population for each tract
def create_map(gdf):
    m = folium.Map(location=[45.5, -122.5], 
                   zoom_start=10, 
                   width='100%')
    cp = folium.Choropleth(
        geo_data=gdf,
        name='Mulntomah County Population by Census Tract',
        data=gdf,
        columns=['TRACT_NO', 'P2_001N'],
        key_on="feature.properties.TRACT_NO",
        fill_color="OrRd", # yellow green blue : YlGnBu
        #bins=[0, 300, 600, 900, 2000, 3000],
        label=['P2_001N'],
        tooltip=['TRACT_NO', 'P2_001N'],
        fill_opacity=0.5,
        line_opacity=0.9,
        legend_name='Population per Census Tract',
    ).add_to(m)

    # This adds tooltips:  https://medium.com/analytics-vidhya/interactive-choropleth-map-in-python-using-folium-4e1479d9e568
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    tooltipadd = folium.features.GeoJson(
        gdf,
        style_function=style_function, 
        control=False,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['TRACT_NO', 'P2_001N'],
            aliases=['TRACT_NO', 'P2_001N'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(tooltipadd)
    m.keep_in_front(tooltipadd)
    folium.LayerControl().add_to(m)
    return m


In [21]:
# create_map(gdf)

Validations that the data is correctly labeled

In [22]:
# # Filter what goes into the map 
# gdf_north = gdf[gdf['LABEL']=='N']
# create_map(gdf_north)

In [58]:
# # Filter what goes into the map 
# gdf_ne = gdf[gdf['LABEL']=='NE']
# create_map(gdf_ne)

In [26]:
# # Filter what goes into the map 
# gdf_se = gdf[gdf['LABEL']=='SE']
# create_map(gdf_se)

In [27]:
# # Filter what goes into the map 
# gdf_nw = gdf[gdf['LABEL']=='NW']
# create_map(gdf_nw)

In [28]:
# # Filter what goes into the map 
# gdf_sw = gdf[gdf['LABEL']=='SW']
# create_map(gdf_sw)

In [52]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Population
# ax = gdf.plot(figsize=(100, 100), 
#                  column='P2_001N', #'P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['P2_001N'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])

In [57]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Census Tract
# ax = gdf.plot(figsize=(50, 50), 
#                  column='P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['TRACT_NO'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])